In [5]:
pwd

'/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car'

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [3]:
!pip install numpy==1.15.4


  Using cached https://files.pythonhosted.org/packages/ff/7f/9d804d2348471c67a7d8b5f84f9bc59fd1cefa148986f2b74552f8573555/numpy-1.15.4-cp36-cp36m-manylinux1_x86_64.whl
ERROR: tensorflow 1.15.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.15.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: astropy 4.0 has requirement numpy>=1.16, but you'll have numpy 1.15.4 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.15.1
    Uninstalling numpy-1.15.1:
      Successfully uninstalled numpy-1.15.1


In [1]:
!pip install -- upgrade tables
!pip install eli5

ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [2]:
!pip install xgboost

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
df = pd.read_hdf('data/car.h5')

In [27]:
df.shape

(106494, 155)

##Feature Engineering

In [0]:
SUFFIX_CAT = '_cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values=df[feat].factorize()[0]

  if SUFFIX_CAT in feat:
   df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [29]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

##DecissionTree

In [34]:
  run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

##RandomForest

In [37]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

##XGBoost

In [42]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate':0.1,
    'seed' : 0
}

run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[18:09:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:10:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:10:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [45]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X,y)

imp = PermutationImportance(m).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[18:24:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1193 ± 0.0034,param_napęd_cat
0.1136 ± 0.0030,param_rok-produkcji_cat
0.1083 ± 0.0045,param_stan_cat
0.0610 ± 0.0023,param_skrzynia-biegów_cat
0.0576 ± 0.0034,param_faktura-vat_cat
0.0496 ± 0.0020,param_moc_cat
0.0275 ± 0.0009,param_marka-pojazdu_cat
0.0241 ± 0.0004,feature_kamera-cofania_cat
0.0211 ± 0.0003,param_typ_cat
0.0174 ± 0.0005,param_pojemność-skokowa_cat


In [0]:
feats = ['param_napęd_cat', 'param_rok-produkcji_cat','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc_cat', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 
'param_typ_cat', 'param_pojemność-skokowa_cat', 'seller_name_cat', 'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat', 'param_wersja_cat', 'param_kod-silnika_cat', 'feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat']

In [47]:
len(feats)

20

In [48]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[18:44:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:44:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [52]:
df['param_rok-produkcji_cat'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, -1, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
       50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75])

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)== 'None' else int(x))

In [54]:
feats = ['param_napęd_cat', 'param_rok-produkcji','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc_cat', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 
'param_typ_cat', 'param_pojemność-skokowa_cat', 'seller_name_cat', 'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat', 'param_wersja_cat', 'param_kod-silnika_cat', 'feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[18:56:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [55]:
df['param_moc'].unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [0]:
df['param_moc']=df['param_moc'].map(lambda x: -1 if str(x)== 'None' else int(x.split(' ')[0]))

In [61]:
feats = ['param_napęd_cat', 'param_rok-produkcji','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 
'param_typ_cat', 'param_pojemność-skokowa_cat', 'seller_name_cat', 'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat', 'param_wersja_cat', 'param_kod-silnika_cat', 'feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:17:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:17:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:17:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [66]:
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa']= df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)== 'None' else int(str(x).split('cm')[0].replace(' ','')))

In [74]:
feats = ['param_napęd_cat', 'param_rok-produkcji','param_stan_cat','param_skrzynia-biegów_cat','param_faktura-vat_cat','param_moc', 'param_marka-pojazdu_cat', 'feature_kamera-cofania_cat', 
'param_typ_cat', 'param_pojemność-skokowa', 'seller_name_cat', 'feature_wspomaganie-kierownicy_cat',
'param_model-pojazdu_cat', 'param_wersja_cat', 'param_kod-silnika_cat', 'feature_system-start-stop_cat','feature_asystent-pasa-ruchu_cat', 'feature_czujniki-parkowania-przednie_cat',
'feature_łopatki-zmiany-biegów_cat', 'feature_regulowane-zawieszenie_cat']

run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:36:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:36:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:36:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)